In [1]:
# first we create and reset a OpenAI Gym environment
try:
    from importlib.resources import files
except ImportError:
    from importlib_resources import files
import gym
import os

env = gym.make(
    "gym_saturation:saturation-v0",
    # for a classical syllogism about Socrates
    problem_list=[
        files("gym_saturation").joinpath(
            os.path.join("resources", "TPTP-mock", "Problems",
                         "TST", "TST003-1.p")
        )
    ],
)
env.reset()
# we can render the environment (that will become the beginning of the proof)
print("starting hypotheses:")
print(env.render("human"))
# our 'age' agent will always select clauses for inference
# in the order they appeared in current proof attempt
action = 0
done = False
while not done:
    observation, reward, done, info = env.step(action)
    action += 1
# SaturationEnv has an additional method
# for extracting only clauses which became parts of the proof
# (some steps were unnecessary to find the proof)
print("refutation proof:")
print(env.tstp_proof)
print(f"number of attempted steps: {action}")

starting hypotheses:
cnf(p_imp_q, hypothesis, ~man(X0) | mortal(X0)).
cnf(p, hypothesis, man(socrates)).
cnf(q, negated_goal, ~mortal(socrates)).
refutation proof:
cnf(xf3d00cf9_85fb_11ec_b77c_9f47bc9de9c1, lemma, mortal(socrates), inference(resolution, [], [p_imp_q, p])).
cnf(xf3d13ca8_85fb_11ec_b657_9f47bc9de9c1, lemma, $false, inference(resolution, [], [q, xf3d00cf9_85fb_11ec_b77c_9f47bc9de9c1])).
number of attempted steps: 6
